### Матричные факторизации

В данной работе вам предстоит познакомиться с практической стороной матричных разложений.
Работа поделена на 4 задания:
1. Вам необходимо реализовать SVD разложения используя SGD на explicit данных
2. Вам необходимо реализовать матричное разложения используя ALS на implicit данных
3. Вам необходимо реализовать матричное разложения используя BPR(pair-wise loss) на implicit данных
4. Вам необходимо реализовать матричное разложения используя WARP(list-wise loss) на implicit данных

Мягкий дедлайн 28 Сентября (пишутся замечания, выставляется оценка, есть возможность исправить до жесткого дедлайна)

Жесткий дедлайн 5 Октября (Итоговая проверка)

In [2]:
import implicit
import pandas as pd
import numpy as np
import scipy.sparse as sp
from tqdm import tqdm
from lightfm.datasets import fetch_movielens

/Library/miniconda3/lib/python3.8/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


В данной работе мы будем работать с explicit датасетом movieLens, в котором представленны пары user_id movie_id и rating выставленный пользователем фильму

Скачать датасет можно по ссылке https://grouplens.org/datasets/movielens/1m/

In [3]:
ratings = pd.read_csv('ratings.dat', delimiter='::', header=None, 
        names=['user_id', 'movie_id', 'rating', 'timestamp'], 
        usecols=['user_id', 'movie_id', 'rating'], engine='python')

In [4]:
movie_info = pd.read_csv('movies.dat', delimiter='::', header=None, 
        names=['movie_id', 'name', 'category'], engine='python')

Explicit данные

In [5]:
ratings

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
...,...,...,...
1000204,6040,1091,1
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4


Для того, чтобы преобразовать текущий датасет в Implicit, давайте считать что позитивная оценка это оценка >=4

In [6]:
implicit_ratings = ratings.loc[(ratings['rating'] >= 4)]

In [7]:
implicit_ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
3,1,3408,4
4,1,2355,5
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4
10,1,595,5
11,1,938,4
12,1,2398,4


Удобнее работать с sparse матричками, давайте преобразуем DataFrame в CSR матрицы

In [8]:
users = implicit_ratings["user_id"]
movies = implicit_ratings["movie_id"]
user_item = sp.coo_matrix((np.ones_like(users), (users, movies)))
user_item_t_csr = user_item.T.tocsr()
user_item_csr = user_item.tocsr()

In [9]:
model = implicit.als.AlternatingLeastSquares(factors=64, iterations=100, calculate_training_loss=True )

В качестве loss здесь всеми любимый RMSE

In [10]:
model.fit(user_item_t_csr)

Построим похожие фильмы по 1 movie_id = Истории игрушек

In [11]:
movie_info.head(5)

,movie_id,name,category
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [12]:
get_similars = lambda item_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                        for x in model.similar_items(item_id)]

Как мы видим, симилары действительно оказались симиларами.

Качество симиларов часто является хорошим способом проверить качество алгоритмов.

P.S. Если хочется поглубже разобраться в том как разные алгоритмы формируют разные латентные пространства, рекомендую загружать полученные вектора в tensorBoard и смотреть на сформированное пространство

In [13]:
get_similars(1, model)

['0    Toy Story (1995)',
 '3045    Toy Story 2 (1999)',
 "2286    Bug's Life, A (1998)",
 '33    Babe (1995)',
 '584    Aladdin (1992)',
 '2315    Babe: Pig in the City (1998)',
 '360    Lion King, The (1994)',
 '1526    Hercules (1997)',
 '1838    Mulan (1998)',
 '2618    Tarzan (1999)']

Давайте теперь построим рекомендации для юзеров

Как мы видим юзеру нравится фантастика, значит и в рекомендациях ожидаем увидеть фантастику

In [14]:
get_user_history = lambda user_id, implicit_ratings : [movie_info[movie_info["movie_id"] == x]["name"].to_string() 
                                            for x in implicit_ratings[implicit_ratings["user_id"] == user_id]["movie_id"]]

In [15]:
get_user_history(4, implicit_ratings)

['3399    Hustler, The (1961)',
 '2882    Fistful of Dollars, A (1964)',
 '1196    Alien (1979)',
 '1023    Die Hard (1988)',
 '257    Star Wars: Episode IV - A New Hope (1977)',
 '1959    Saving Private Ryan (1998)',
 '476    Jurassic Park (1993)',
 '1180    Raiders of the Lost Ark (1981)',
 '1885    Rocky (1976)',
 '1081    E.T. the Extra-Terrestrial (1982)',
 '3349    Thelma & Louise (1991)',
 '3633    Mad Max (1979)',
 '2297    King Kong (1933)',
 '1366    Jaws (1975)',
 '1183    Good, The Bad and The Ugly, The (1966)',
 '2623    Run Lola Run (Lola rennt) (1998)',
 '2878    Goldfinger (1964)',
 '1220    Terminator, The (1984)']

Получилось! 

Мы действительно порекомендовали пользователю фантастику и боевики, более того встречаются продолжения тех фильмов, которые он высоко оценил

In [16]:
get_recommendations = lambda user_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                               for x in model.recommend(user_id, user_item_csr)]

In [17]:
get_recommendations(4, model)

['585    Terminator 2: Judgment Day (1991)',
 '1271    Indiana Jones and the Last Crusade (1989)',
 '1182    Aliens (1986)',
 '2502    Matrix, The (1999)',
 '1284    Butch Cassidy and the Sundance Kid (1969)',
 '1178    Star Wars: Episode V - The Empire Strikes Back...',
 '3402    Close Encounters of the Third Kind (1977)',
 '847    Godfather, The (1972)',
 '2460    Planet of the Apes (1968)',
 '2880    Dr. No (1962)']

Теперь ваша очередь реализовать самые популярные алгоритмы матричных разложений

Что будет оцениваться:
1. Корректность алгоритма
2. Качество получившихся симиларов
3. Качество итоговых рекомендаций для юзера

In [18]:
get_recommendations = lambda user_id, model, k : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                               for x in model.recommend(user_id, k)]

### Задание 1. Не использую готовые решения, реализовать SVD разложение используя SGD на explicit данных

In [19]:
class Base:
    def __init__(self, df=ratings, n_users=ratings["user_id"].max(), n_items=ratings["movie_id"].max(), 
                 K=64, lr=0.01, lam=0.01, max_steps=int(100e6), step_check=int(1e5), eps=1e-7, init_bias=True):
        
        self.df = df.copy()
        self.data = self.df.to_numpy()

        self.n_nonzero = self.data.shape[0]
        self.n_users, self.n_items = n_users, n_items
        self.unique_items = self.df["movie_id"].unique()
        self.unique_users = self.df["user_id"].unique()   
        
        self.K = K # latent size
        self.lr = lr # learning rate 
        self.lam = lam # regularization parameter
        self.eps = eps # convergence threshold
        self.max_steps = max_steps # max_steps
        self.step_check = step_check # print every steps
        
        self.W = np.random.uniform(0, 1/np.sqrt(self.K), size=(self.n_users, self.K))  
        self.H = np.random.uniform(0, 1/np.sqrt(self.K), size=(self.n_items, self.K))

        self.V = None
        
        if init_bias:
            
            #self.bias = self.data[:, 2].mean()
            self.bias =user_item.mean()
            self.W_bias = np.array(user_item.mean(axis=1)[1:, :])
            self.H_bias = np.array(user_item.mean(axis=0)[:, 1:]).reshape(-1, 1)

                
    def check_convergence(self, step, show=True):
        """
        Calculate RMSE and check the convergence condition   
        """
        R_predicted = self.V[self.data[:, 0] - 1, self.data[:, 1] - 1]
        R_true = self.data[:, 2]
        RMSE = np.linalg.norm(R_predicted - R_true)/self.n_nonzero
        if show:
            print(f"[{step}] RMSE: {RMSE:.4}")
        return RMSE < self.eps 
            
        
    def similar_items(self, item_id, k=10):
        """
        Get similar items for useritem_id according to fitted embeddings 
        """
        ratings = [(other_item_id, np.linalg.norm(self.H[item_id - 1] - self.H[other_item_id - 1]))
                   for other_item_id in self.unique_items]
        items_ratings = sorted(ratings, key=lambda x: x[1]) 
        return items_ratings[:k]
        
    def recommend(self, user_id, k=10):
        """
        Recommend users new items which he didn't look before 
        """
        new_items_ids = list(set(self.unique_items) - set(self.df.loc[self.df["user_id"] == user_id]["movie_id"]))
        new_items_ratings = self.V[user_id - 1][np.array(new_items_ids) - 1]
        items_ratings = sorted(list(zip(new_items_ids, new_items_ratings)), key=lambda x: x[1], reverse=True) 
        return items_ratings[:k]
    

In [19]:
class SVD(Base):
    def __init__(self, df=ratings, n_users=ratings["user_id"].max(), n_items=ratings["movie_id"].max(), 
                 K=64, lr=0.01, lam=1e-5, max_steps=int(100e6), step_check=int(1e5), eps=1e-7, init_bias=True, lam_bias=1e-6):
        Base.__init__(self, df, n_users, n_items, K, lr, lam, max_steps, step_check, eps, init_bias)
        self.lam_bias = lam_bias
        self.recommendation_mat()
        
    def recommendation_mat(self):
        self.V = self.bias + self.W_bias + self.H_bias.T + self.W @ self.H.T
        
    def fit(self):
        """
        Fit SVD  
        """
        for step in range(self.max_steps + 1):
            
            if step % self.step_check == 0:
                self.recommendation_mat()
                if self.check_convergence(step, show=True):
                    self.recommendation_mat()
                    break 
                
            i, j, rating = self.data[np.random.randint(self.n_nonzero), :]
            i-=1
            j-=1
            error = (self.bias + self.W_bias[i] + self.H_bias[j] + self.W[i, ]@self.H[j,].T) - rating
            
            self.bias -= self.lr * error
            
            self.W_bias[i, ] -= self.lr * (error + self.lam_bias * self.W_bias[i, ])
            self.H_bias[j, ] -= self.lr * (error + self.lam_bias * self.H_bias[j, ])  
            
            W_i = self.W[i, ][:]
            self.W[i, ] -= self.lr * (error * self.H[j, ] + self.lam * self.W[i, ]) 
            self.H[j, ] -= self.lr * (error * W_i + self.lam * self.H[j, ])  
              
            
        self.recommendation_mat()

In [26]:
svd = SVD(df=ratings, n_users=ratings["user_id"].max(), n_items=ratings["movie_id"].max(), 
          K=30, lr=0.01, lam=1e-5, max_steps=int(2e7), step_check=int(2e6), eps=1e-6, init_bias=True, lam_bias=1e-6)
svd.fit()

[0] RMSE: 0.00335
[2000000] RMSE: 0.0008988
[4000000] RMSE: 0.0008679
[6000000] RMSE: 0.0008137
[8000000] RMSE: 0.0007767
[10000000] RMSE: 0.0007499
[12000000] RMSE: 0.0007366
[14000000] RMSE: 0.0007206
[16000000] RMSE: 0.0007113
[18000000] RMSE: 0.0007084
[20000000] RMSE: 0.0006989


In [27]:
get_similars(1, svd)

['0    Toy Story (1995)',
 '3045    Toy Story 2 (1999)',
 "2286    Bug's Life, A (1998)",
 '1838    Mulan (1998)',
 '1985    Honey, I Shrunk the Kids (1989)',
 '360    Lion King, The (1994)',
 '1526    Hercules (1997)',
 '1375    Sneakers (1992)',
 '259    Little Princess, A (1995)',
 '3546    Dinosaur (2000)']

In [28]:
get_user_history(4, implicit_ratings)

['3399    Hustler, The (1961)',
 '2882    Fistful of Dollars, A (1964)',
 '1196    Alien (1979)',
 '1023    Die Hard (1988)',
 '257    Star Wars: Episode IV - A New Hope (1977)',
 '1959    Saving Private Ryan (1998)',
 '476    Jurassic Park (1993)',
 '1180    Raiders of the Lost Ark (1981)',
 '1885    Rocky (1976)',
 '1081    E.T. the Extra-Terrestrial (1982)',
 '3349    Thelma & Louise (1991)',
 '3633    Mad Max (1979)',
 '2297    King Kong (1933)',
 '1366    Jaws (1975)',
 '1183    Good, The Bad and The Ugly, The (1966)',
 '2623    Run Lola Run (Lola rennt) (1998)',
 '2878    Goldfinger (1964)',
 '1220    Terminator, The (1984)']

In [31]:
get_recommendations(4, svd, 10)

["1176    One Flew Over the Cuckoo's Nest (1975)",
 '1575    L.A. Confidential (1997)',
 '847    Godfather, The (1972)',
 '1211    Annie Hall (1977)',
 '1203    Godfather: Part II, The (1974)',
 '1189    To Kill a Mockingbird (1962)',
 '1232    Chinatown (1974)',
 '315    Shawshank Redemption, The (1994)',
 '2255    Life Is Beautiful (La Vita � bella) (1997)',
 '896    North by Northwest (1959)']

In [34]:
get_recommendations(100, svd, 10)

['1022    Sound of Music, The (1965)',
 '2735    Christmas Story, A (1983)',
 '1878    West Side Story (1961)',
 "887    Singin' in the Rain (1952)",
 '2255    Life Is Beautiful (La Vita � bella) (1997)',
 '898    Some Like It Hot (1959)',
 '1340    Paradise Lost: The Child Murders at Robin Hood...',
 '1276    Room with a View, A (1986)',
 '2928    Being John Malkovich (1999)',
 '1058    Willy Wonka and the Chocolate Factory (1971)']

### Задание 2. Не использую готовые решения, реализовать матричное разложение используя ALS на implicit данных

In [27]:
class ALS(Base):
    def __init__(self, df=implicit_ratings, n_users=implicit_ratings["user_id"].max(), n_items=implicit_ratings["movie_id"].max(), 
                 K=30, lr=1e-3, lam=1e-3, max_steps=int(100), step_check=int(10), eps=1e-6, init_bias=False):
        Base.__init__(self, df, n_users, n_items, K, lr, lam, max_steps, step_check, eps, init_bias)
        self.recommendation_mat()

    def recommendation_mat(self):
        self.V = self.W @ self.H.T
           
    def fit(self):
        """
        Fit ALS 
        """
        for step in range(self.max_steps + 1):
            self.recommendation_mat()
            
            if step % self.step_check == 0:
                if self.check_convergence(step, show=True):
                    self.recommendation_mat()
                    break 
                    
            error = self.V[:]
            error[self.data[:, 0] - 1, self.data[:, 1] - 1] -= self.data[:, 2]
            
            if step % 2 == 0:
                self.W -= self.lr * (error@self.H + self.lam * self.W)
            else:
                self.H -= self.lr * (error.T@self.W + self.lam * self.H)   
                
        self.recommendation_mat()
            

In [28]:
als = ALS()
als.fit()

[0] RMSE: 0.005501
[10] RMSE: 0.004802
[20] RMSE: 0.004575
[30] RMSE: 0.004317
[40] RMSE: 0.004178
[50] RMSE: 0.004067
[60] RMSE: 0.004004
[70] RMSE: 0.003974
[80] RMSE: 0.003961
[90] RMSE: 0.003953
[100] RMSE: 0.003949


In [29]:
get_similars(1, als)

['0    Toy Story (1995)',
 '3045    Toy Story 2 (1999)',
 '584    Aladdin (1992)',
 "2286    Bug's Life, A (1998)",
 '33    Babe (1995)',
 '360    Lion King, The (1994)',
 '591    Beauty and the Beast (1991)',
 '1838    Mulan (1998)',
 '2252    Pleasantville (1998)',
 '1245    Groundhog Day (1993)']

In [30]:
get_recommendations(4, als)

['1178    Star Wars: Episode V - The Empire Strikes Back...',
 '1192    Star Wars: Episode VI - Return of the Jedi (1983)',
 '847    Godfather, The (1972)',
 '1182    Aliens (1986)',
 '1271    Indiana Jones and the Last Crusade (1989)',
 '585    Terminator 2: Judgment Day (1991)',
 '1203    Godfather: Part II, The (1974)',
 '453    Fugitive, The (1993)',
 '1284    Butch Cassidy and the Sundance Kid (1969)',
 '3458    Predator (1987)']

In [31]:
get_user_history(4, implicit_ratings)

['3399    Hustler, The (1961)',
 '2882    Fistful of Dollars, A (1964)',
 '1196    Alien (1979)',
 '1023    Die Hard (1988)',
 '257    Star Wars: Episode IV - A New Hope (1977)',
 '1959    Saving Private Ryan (1998)',
 '476    Jurassic Park (1993)',
 '1180    Raiders of the Lost Ark (1981)',
 '1885    Rocky (1976)',
 '1081    E.T. the Extra-Terrestrial (1982)',
 '3349    Thelma & Louise (1991)',
 '3633    Mad Max (1979)',
 '2297    King Kong (1933)',
 '1366    Jaws (1975)',
 '1183    Good, The Bad and The Ugly, The (1966)',
 '2623    Run Lola Run (Lola rennt) (1998)',
 '2878    Goldfinger (1964)',
 '1220    Terminator, The (1984)']

### Задание 3. Не использую готовые решения, реализовать матричное разложение BPR на implicit данных

In [42]:
class BPR(Base):
    def __init__(self, df=implicit_ratings, n_users=implicit_ratings["user_id"].max(), n_items=implicit_ratings["movie_id"].max(), 
                 K=30, lr=1e-1, lam=1e-3, max_steps=int(1e6), step_check=int(1e4), eps=1e-6, init_bias=False, sample=5):
        Base.__init__(self, df, n_users, n_items, K, lr, lam, max_steps, step_check, eps, init_bias)
        self.sample = sample
        self.recommendation_mat()

    def recommendation_mat(self):
        self.V = self.W @ self.H.T
        
    def compute_pos_neg(self):
        self.user_pos_neg = {}
        for u in tqdm(self.unique_users):         
            user_pos_items = self.data[self.data[:, 0] == u][:, 1]
            user_neg_items = list(set(self.unique_items) - set(user_pos_items))
            self.user_pos_neg[u] = (user_pos_items, user_neg_items)
           
    def fit(self):
        """
        Fit BPR
        """
        self.compute_pos_neg() 
        for step in range(self.max_steps + 1):
            
            if step % self.step_check == 0:
                self.recommendation_mat()
                if self.check_convergence(step, show=True):
                    self.recommendation_mat()
                    break 
                   
            for u in tqdm(self.unique_users):
                user_pos_items, user_neg_items = self.user_pos_neg[u]
                u -= 1
                for i in user_pos_items:
                    i -= 1
                    neg_sample = np.random.choice(user_neg_items, size=self.sample, replace=False)
                    
                    for j in neg_sample:                        
                        j -= 1
                        r_ui = self.W[u]@self.H[i].T
                        r_uj = self.W[u]@self.H[j].T

                        L =  1 / (1 + np.exp(r_ui - r_uj))

                        W_u = self.W[u][:]

                        self.W[u] += self.lr * (L * (self.H[i] - self.H[j]) - self.lam * self.W[u])
                        self.H[i] += self.lr * (L * W_u - self.lam * self.H[i])
                        self.H[j] += self.lr * (L * (-W_u) - self.lam * self.H[j])                     
                            
        self.recommendation_mat()

In [43]:
bpr = BPR(df=implicit_ratings, n_users=implicit_ratings["user_id"].max(), n_items=implicit_ratings["movie_id"].max(), 
          K=100, lr=0.01, lam=0.0001, max_steps=int(5), step_check=int(1), eps=1e-6, init_bias=True)
bpr.fit()

  0%|          | 5/6038 [00:00<02:26, 41.20it/s]

[0] RMSE: 0.005501


  0%|          | 3/6038 [00:00<03:56, 25.50it/s]

[1] RMSE: 0.003055


  0%|          | 4/6038 [00:00<02:31, 39.78it/s]

...

  0%|          | 4/6038 [00:00<02:37, 38.28it/s]

[4] RMSE: 0.002395


  0%|          | 4/6038 [00:00<02:40, 37.56it/s]

[5] RMSE: 0.002277


100%|██████████| 6038/6038 [04:35<00:00, 21.88it/s]


In [44]:
get_similars(1, bpr)

['0    Toy Story (1995)',
 '3045    Toy Story 2 (1999)',
 '1179    Princess Bride, The (1987)',
 '1245    Groundhog Day (1993)',
 "2286    Bug's Life, A (1998)",
 '1250    Back to the Future (1985)',
 '33    Babe (1995)',
 '1539    Men in Black (1997)',
 '584    Aladdin (1992)',
 '2918    Who Framed Roger Rabbit? (1988)']

In [45]:
get_recommendations(4, bpr)

['1178    Star Wars: Episode V - The Empire Strikes Back...',
 '2502    Matrix, The (1999)',
 '2789    American Beauty (1999)',
 '847    Godfather, The (1972)',
 '585    Terminator 2: Judgment Day (1991)',
 '1192    Star Wars: Episode VI - Return of the Jedi (1983)',
 '2693    Sixth Sense, The (1999)',
 '589    Silence of the Lambs, The (1991)',
 '604    Fargo (1996)',
 '1575    L.A. Confidential (1997)']

### Задание 4. Не использую готовые решения, реализовать матричное разложение WARP на implicit данных

In [20]:
class WARP(Base):
    def __init__(self, df=implicit_ratings, n_users=implicit_ratings["user_id"].max(), n_items=implicit_ratings["movie_id"].max(), 
                 K=30, lr=1e-1, lam=1e-3, max_steps=int(1e6), step_check=int(1e4), eps=1e-6, init_bias=True, sample=10):
        Base.__init__(self, df, n_users, n_items, K, lr, lam, max_steps, step_check, eps, init_bias)
       
        self.sample = sample
        self.compute_pos_neg() 
        self.recommendation_mat()

    def recommendation_mat(self):
        self.V = self.W @ self.H.T
        
    def compute_pos_neg(self):
        self.user_pos_neg = {}
        for u in tqdm(self.unique_users):         
            user_pos_items = self.data[self.data[:, 0] == u][:, 1]
            user_neg_items = list(set(self.unique_items) - set(user_pos_items))
            self.user_pos_neg[u] = (user_pos_items, user_neg_items)
            
    def r(self, user, item):
        return self.W[user]@self.H[item].T
    
    def best_neg_sample(self, user, pos_item, user_neg_items, rank = 0):
        r_ui = self.r(user, pos_item)
        j_s = np.random.choice(user_neg_items, size=min(len(user_neg_items), self.sample), replace=False) - 1
        r_ujs = self.r(user, j_s)
        for r_uj, j in zip(r_ujs, j_s):
            rank += 1
            if r_ui < r_uj + 1:
                break 
        return j, np.log(min(len(user_neg_items), self.sample) / rank) 

    def fit(self):
        """
        Fit WARP
        """
        for step in range(self.max_steps):
           
            if step % self.step_check == 0:
                self.recommendation_mat()
                if self.check_convergence(step, show=True):
                    self.recommendation_mat()
                    break
            if step == 3:
                self.lr = 0.001
                   
            for u in tqdm(self.unique_users):
                user_pos_items, user_neg_items = self.user_pos_neg[u]
                u -= 1
                for i in user_pos_items:
                    i -= 1
        
                    j, weight = self.best_neg_sample(u, i, user_neg_items)

                    W_u = self.W[u][:]

                    self.W[u] += self.lr * (weight * (self.H[i] - self.H[j]) - self.lam * self.W[u])
                    self.H[i] += self.lr * (weight * W_u - self.lam * self.H[i])
                    self.H[j] += self.lr * (weight * (-W_u) - self.lam * self.H[j]) 
                            
        self.recommendation_mat()
            

In [21]:
warp = WARP(df=implicit_ratings, n_users=implicit_ratings["user_id"].max(), n_items=implicit_ratings["movie_id"].max(), 
          K=64, lr=0.01, lam=1e-3, max_steps=int(12), step_check=int(1), eps=1e-6, init_bias=False, sample=50)
warp.fit()

  0%|          | 5/6038 [00:00<02:50, 35.42it/s]

[0] RMSE: 0.005501


  0%|          | 5/6038 [00:00<02:39, 37.82it/s]

[1] RMSE: 0.003079


  0%|          | 5/6038 [00:00<02:18, 43.57it/s]

...

  0%|          | 5/6038 [00:00<02:25, 41.36it/s]

[10] RMSE: 0.002256


  0%|          | 5/6038 [00:00<02:35, 38.90it/s]

[11] RMSE: 0.00224


100%|██████████| 6038/6038 [03:48<00:00, 26.41it/s]


In [23]:
get_similars(1, warp)

['0    Toy Story (1995)',
 '2225    Antz (1998)',
 '3045    Toy Story 2 (1999)',
 "2286    Bug's Life, A (1998)",
 '2618    Tarzan (1999)',
 '1838    Mulan (1998)',
 '1526    Hercules (1997)',
 '2014    Muppet Christmas Carol, The (1992)',
 '2315    Babe: Pig in the City (1998)',
 '3090    Fantasia 2000 (1999)']

In [24]:
get_recommendations(4, warp, 10)

['585    Terminator 2: Judgment Day (1991)',
 '1178    Star Wars: Episode V - The Empire Strikes Back...',
 '847    Godfather, The (1972)',
 '2502    Matrix, The (1999)',
 '1182    Aliens (1986)',
 '1568    Hunt for Red October, The (1990)',
 '537    Blade Runner (1982)',
 '1353    Star Trek: The Wrath of Khan (1982)',
 '1284    Butch Cassidy and the Sundance Kid (1969)',
 '108    Braveheart (1995)']